In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

We can use Hypergraph features for various predictive tasks:

In [2]:
from convokit import PairedPrediction

In [3]:
import convokit
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
# os.chdir('..')

In [5]:
# os.chdir('reddit-corpus')

In [6]:
os.listdir()

['LICENSE.md',
 'convokit',
 '.DS_Store',
 'hyperconvo_feats.p',
 'requirements.txt',
 'dist',
 'Makefile',
 'website',
 'datasets',
 'tests',
 'README.md',
 'setup.py',
 '.gitignore',
 'CONTRIBUTING.md',
 'examples',
 'doc',
 'venv-triadmotif',
 'convokit.egg-info',
 'venv',
 '.git',
 '.idea']

In [7]:
corpus = convokit.Corpus(filename=convokit.download('reddit-corpus'))
# corpus = convokit.Corpus(filename='reddit-corpus')

Dataset already exists at /Users/calebchiam/.convokit/downloads/reddit-corpus


In [8]:
hc = convokit.HyperConvo(prefix_len=10, min_thread_len=10, include_root=False)
hc.fit_transform(corpus)

In [9]:
remake_cache = True
if remake_cache:
    with open("hyperconvo_feats.p", "wb") as f:
        hyperconvo_feats = {}
        for convo in corpus.iter_conversations():
            hyperconvo_feats.update(convo.meta["hyperconvo"])
        pickle.dump(hyperconvo_feats, f)
else:
    with open("hyperconvo_feats.p", "rb") as f:
        hyperconvo_feats = pickle.load(f)

In [10]:
threads = corpus.utterance_threads(include_root=False)

In [11]:
# Use only the first 10 comments in each thread
thread_pfxs = corpus.utterance_threads(prefix_len=10, include_root=False)

In [12]:
len(threads)

100000

In [13]:
corpus.print_summary_stats()

Number of Users: 521777
Number of Utterances: 2004262
Number of Conversations: 84979


In [14]:
from collections import defaultdict
thread_roots_by_self_post = defaultdict(list)
for top_level_comment, thread in threads.items():
    rt = thread[next(iter(thread))].root
    thread_roots_by_self_post[rt].append(top_level_comment)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
hyperconvo_feats = hc.retrieve_feats(corpus)

In [17]:
path_stats = hc.retrieve_motif_pathway_stats(corpus)

In [18]:
motif_counts = hc.retrieve_motif_counts(corpus)

In [19]:
threads_motifs = hc.retrieve_motifs(corpus)

In [20]:
latent_motif_count = {thread_id: hc._latent_motif_count(motif_dict, trans=False)[0] for thread_id, motif_dict in threads_motifs.items()}

In [21]:
hyperconv_df = pd.DataFrame.from_dict(hyperconvo_feats, orient='index')
hyperconv_feat_names = list(hyperconv_df.columns)

In [48]:
path_stats_df = pd.DataFrame.from_dict(path_stats, orient='index')
columns = ['PATH-'+', '.join(filter(lambda x: type(x) == str, col)).strip() for col in path_stats_df.columns.values]
path_stats_df.columns = columns
path_feat_names = list(path_stats_df.columns)

In [23]:
motif_counts_df = pd.DataFrame.from_dict(motif_counts, orient='index')
motif_feat_names = list(motif_counts_df.columns)

In [24]:
latentmotif_df = pd.DataFrame.from_dict(latent_motif_count, orient='index')
latentmotif_df.columns = ['LATENT_'+c for c in latentmotif_df.columns]
latent_motif_feat_names = list(latentmotif_df.columns)

In [25]:
def get_num_users(thread):
    return len(set(utt.user.name for utt in thread.values()))

thread_to_usercount = dict()
for thread_id in thread_pfxs:
    thread_to_usercount[thread_id] = {"num_users": get_num_users(thread_pfxs[thread_id])}

In [26]:
num_users_df = pd.DataFrame.from_dict(thread_to_usercount, orient='index')
num_users_feat = ['num_users']

In [49]:
feats_df = pd.concat([hyperconv_df, path_stats_df, motif_counts_df, latentmotif_df, num_users_df], axis=1)

In [30]:
# first generate positive and negative examples based on task
import random
def generate_pos_neg(task: str, post_to_thread_obj, threads, thread_pfxs):
    pos, neg = [], []
    if task == "comment-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos = [root for root in thread_roots if len(threads[root]) >= 15]
            has_neg = [root for root in thread_roots if len(threads[root]) == 10]
            
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    elif task == "commenter-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos, has_neg = [], []
            for root in thread_roots:
                if len(threads[root]) >= 20:
                    if len(set(c.user.name for c in threads[root].values())) >= \
                        len(set(c.user.name for c in thread_pfxs[root].values())) * 2:
                            has_pos.append(root)
                    else:
                        has_neg.append(root)
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    print("- {} positive, {} negative pts for {} task".format(len(pos), len(neg), task))
    
    return pos, neg


In [31]:
pos_comment_growth, neg_comment_growth = generate_pos_neg("comment-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 1827 positive, 1827 negative pts for comment-growth task


In [32]:
pos_commenter_growth, neg_commenter_growth = generate_pos_neg("commenter-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 849 positive, 849 negative pts for commenter-growth task


In [50]:
random.seed(2019)
for task in ["comment-growth", "commenter-growth"]: #, "post-deleted", "user-deleted"
    print("TASK: {}\n".format(task))
    
    pos, neg = generate_pos_neg(task, thread_roots_by_self_post, threads, thread_pfxs)
    for feature_set, name in [(hyperconv_feat_names, "hyperconvo"),
                        (hyperconv_feat_names + num_users_feat, "hyperconv-usercount"),
                        (latent_motif_feat_names, "latentmotif"),
                        (latent_motif_feat_names + num_users_feat, "latentmotif-usercount"),
                        (path_feat_names, "motifpaths"),
                        (path_feat_names + num_users_feat, "motifpaths-usercount"),
                        (hyperconv_feat_names + motif_feat_names, "hyperconv-motif"),
                        (hyperconv_feat_names + path_feat_names, "hyperconv-paths"),
                        (hyperconv_feat_names + latent_motif_feat_names, "hyperconv-latent"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names + latent_motif_feat_names, "hyperconvo-motifall"),
                        (num_users_feat, "usercount")
                       ]:
        pp = PairedPrediction()
        print("Feature set: {}".format(name))
        pp.fit_predict(feats_df[feature_set], pos, neg, test_size=0.2)
        pp.print_extreme_coefs(feature_set, num_features=5)

TASK: comment-growth

- 1827 positive, 1827 negative pts for comment-growth task
Feature set: hyperconvo
Excluded 49 data point(s) that contained NaN values.
Train accuracy: 0.6456
Test accuracy: 0.5506

TOP 5 FEATURES
prop-nonzero[indegree over C->C mid-thread responses]: 1.245
mean[outdegree over C->c responses]: 0.351
mean[outdegree over C->C responses]: 0.351
mean[indegree over C->C responses]: 0.351
norm.max[outdegree over C->c mid-thread responses]: 0.335

BOTTOM 5 FEATURES
max[indegree over C->C responses]: -0.375
entropy[indegree over C->C mid-thread responses]: -0.376
entropy[indegree over C->C responses]: -0.404
mean-nonzero[indegree over C->C responses]: -0.520
prop-nonzero[indegree over C->C responses]: -1.022

Feature set: hyperconv-usercount
Excluded 49 data point(s) that contained NaN values.
Train accuracy: 0.6456
Test accuracy: 0.5506

TOP 5 FEATURES
prop-nonzero[indegree over C->C mid-thread responses]: 1.246
mean[outdegree over C->c responses]: 0.351
mean[outdegree o

Excluded 20 data point(s) that contained NaN values.
Train accuracy: 0.6425
Test accuracy: 0.4940

TOP 5 FEATURES
entropy[outdegree over C->c mid-thread responses]: 0.578
entropy[outdegree over C->C mid-thread responses]: 0.578
norm.max[outdegree over c->c mid-thread responses]: 0.540
norm.2nd-largest[outdegree over c->c mid-thread responses]: 0.534
entropy[indegree over C->C mid-thread responses]: 0.442

BOTTOM 5 FEATURES
2nd-largest / max[indegree over C->c responses]: -0.458
max[indegree over C->C mid-thread responses]: -0.527
2nd-largest / max[indegree over C->C mid-thread responses]: -0.546
prop-multiple[indegree over c->c mid-thread responses]: -0.619
prop-multiple[indegree over C->c mid-thread responses]: -0.619

Feature set: latentmotif
Train accuracy: 0.5361
Test accuracy: 0.4882

TOP 5 FEATURES
LATENT_INCOMING_1TO3_TRIADS: 0.451
LATENT_SINGLE_EDGE_TRIADS: 0.292
LATENT_DIRECTED_CYCLE_TRIADS: 0.118
LATENT_INCOMING_2TO3_TRIADS: 0.065
LATENT_INCOMING_TRIADS: 0.007

BOTTOM 5 FEATU

In [ ]:
pos, neg = generate_pos_neg("comment-growth", thread_roots_by_self_post, threads, thread_pfxs)

In [ ]:
threads_text = dict()
for thread_id in pos + neg:
    threads_text[thread_id] = {"text": " ".join(utt.text for utt in thread_pfxs[thread_id].values())}

In [ ]:
text_df = pd.DataFrame.from_dict(threads_text).T

In [ ]:
# First split the pos + neg pairs in train, test sets

In [ ]:
from sklearn.model_selection import train_test_split
pos_neg_train, pos_neg_test = train_test_split(list(zip(pos, neg)), test_size=0.2, random_state=42)

In [ ]:
pos_train = [x[0] for x in pos_neg_train]
neg_train = [x[1] for x in pos_neg_train] 
pos_test = [x[0] for x in pos_neg_test]
neg_test = [x[1] for x in pos_neg_test]

In [ ]:
# BOW vectorizer on train dataset

In [ ]:
train_text_df = text_df.loc[pos_train + neg_train]

In [ ]:
test_text_df = text_df.loc[pos_test + neg_test]

In [ ]:
len(test_text_df)

In [ ]:
cv = CountVectorizer(min_df=0.15, max_df=0.7)

In [ ]:
train_text_arr = cv.fit_transform(train_text_df['text'])

In [ ]:
train_text_transform_df = pd.DataFrame(train_text_arr.toarray(), columns=cv.get_feature_names(), index=train_text_df.index)

In [ ]:
train_text_transform_df.head()

In [ ]:
test_text_arr = cv.transform(test_text_df['text'])
test_text_transform_df = pd.DataFrame(test_text_arr.toarray(), columns=cv.get_feature_names(), index=test_text_df.index)   


In [ ]:
from pandas import DataFrame
def _generate_paired_X_y(feats: DataFrame, pos_ids, neg_ids):

    X, y = [], []
    flip = True

    excluded = 0
    for idx in range(len(pos_ids)):
        pos_feats = np.array(feats.loc[pos_ids[idx]])
        neg_feats = np.array(feats.loc[neg_ids[idx]])

        if (np.isnan(pos_feats).any() or np.isnan(neg_feats).any()):
            excluded += 1
            continue

        if flip:
            y.append(1)
            diff = pos_feats - neg_feats
        else:
            y.append(0)
            diff = neg_feats - pos_feats

        X.append(diff)
        flip = not flip

    if excluded > 0:
        print("Excluded {} data point(s) that contained NaN values.".format(excluded))

    return np.array(X), np.array(y)

In [ ]:
X_train, y_train = _generate_paired_X_y(train_text_transform_df, pos_train, neg_train)

In [ ]:
X_test, y_test = _generate_paired_X_y(test_text_transform_df, pos_test, neg_test)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])       

In [ ]:
X_train.shape

In [ ]:
clf = clf.fit(X_train, y_train)

In [ ]:
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("- {}: {:.4f} train, {:.4f} test".format("cumulative_bow", train_acc, test_acc))

In [ ]:
def print_extreme_coefs(clf, feature_names, num_features: int = 5):
    coefs = clf.named_steps['logreg'].coef_[0].tolist()

    assert len(feature_names) == len(coefs)

    feats_coefs = sorted(list(zip(feature_names, coefs)), key=lambda x: x[1], reverse=True)

    print()
    print("TOP {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[:num_features]:
        print("{}: {:.3f}".format(ft, coef))
    print()
    print("BOTTOM {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[-num_features:]:
        print("{}: {:.3f}".format(ft, coef))
    print()

In [ ]:
print_extreme_coefs(clf, cv.get_feature_names(), num_features=20)

In [ ]:
import random
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split

random.seed(2019)

for task in ["comment-growth", "commenter-growth"]: #, "post-deleted", "user-deleted"
    print("TASK: {}\n".format(task))
    
    pos, neg = generate_pos_neg(task, thread_roots_by_self_post, threads, thread_pfxs)

    X, y = generate_paired_features(hyperconvo_feats, pos, neg)
    X_motifcnt, y_motifcnt = generate_paired_features(motif_counts, pos, neg)
    X_latent, y_latent = generate_paired_features(latent_motif_count, pos, neg)
    X_path, y_path = generate_paired_features(path_stats, pos, neg)
    X_hcmotif, y_hcmotif = generate_paired_features(hyperconv_motif, pos, neg)
    X_hcpath, y_hcpath = generate_paired_features(hyperconv_paths, pos, neg)
    X_hclatent, y_hclatent = generate_paired_features(hyperconv_latent, pos, neg)
    X_all, y_all = generate_paired_features(hyperconv_motifall, pos, neg)
    for X, y, feats, name in [(X, y, hyperconvo_feats, "hyperconv"),
                       (X_motifcnt, y_motifcnt, motif_counts, "motifcount"),
                       (X_latent, y_latent, latent_motif_count, "latentmotif"),
                       (X_path, y_path, path_stats, "motifpaths"),
                       (X_hcmotif, y_hcmotif, hyperconv_motif, "hyperconv-motif"),
                       (X_hcpath, y_hcpath, hyperconv_paths, "hyperconv-paths"),
                       (X_hclatent, y_hclatent, hyperconv_latent, "hyperconv-latent"),
                       (X_all, y_all, hyperconv_motifall, "hyperconvo-motifall")
                      ]:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])
        clf.fit(X_train, y_train)

        train_acc = clf.score(X_train, y_train)
        test_acc = clf.score(X_test, y_test)
        print("- {}: {:.4f} train, {:.4f} test".format(name, train_acc, test_acc))
        print_extreme_coefs(clf, feats)